# Import Data

In [1]:
import numpy as np
from netCDF4 import Dataset
directory="/Users/lambda/Documents/Code/CaliforniaDrought2016/Data/"
#file=directory+"Pressure/hgt.mon.mean.nc"
#nc=Dataset(file,'r')
#pressure=nc.variables["hgt"][0:816,7,0:48,48:112]
file=directory+"Pressure/slp.mon.mean.nc"
nc=Dataset(file,'r')
#pressure=nc.variables["slp"][0:815,0:48,48:112]
pressure=nc.variables["slp"][0:816,0:48,48:112]
lat=nc.variables["lat"][:]
lon=nc.variables["lon"][:]
from numpy import loadtxt
CAMonthTotal= loadtxt(directory+'Precipitation/CAMonthTotal.txt')
ORMonthTotal= loadtxt(directory+'Precipitation/ORMonthTotal.txt')
WCMonthTotal= loadtxt(directory+'Precipitation/WCMonthTotal.txt')
CAMonthTotal=CAMonthTotal.reshape((816, 1))
ORMonthTotal=ORMonthTotal.reshape((816, 1))
WCMonthTotal=WCMonthTotal.reshape((816, 1))
CAMonthTotal=CAMonthTotal[0:816,:]
rlat=48
rlon=64

In [2]:
y_data=CAMonthTotal
x=pressure[0:3,:,:]
y=y_data[0:3,:]
for i in range(len(CAMonthTotal)/12):
    x=np.append(x,pressure[10+12*i:15+12*i,:,:],axis=0)
    y=np.append(y,y_data[10+12*i:15+12*i,:],axis=0)
l=len(x)
x_data=x.flatten()
y_data=y
    
x_data=np.reshape((x_data-np.mean(x_data))/np.std(x_data),(l,rlat,rlon,1))
y_data=(y_data-np.mean(y_data))/np.std(y_data)

In [3]:
type(y_data)

numpy.ndarray

# Set Computing Graph

### HyperParameters

In [10]:
ptraining=240
ctraining=[]
ctest=[]
#Convolution Layer Parameters
wc1=5
nc1=3
wc2=5 
nc2=1
wp1=2
wp2=2
fl=(rlat-wc1-wc2-wp1-wp2+4)*(rlon-wc1-wc2-wp1-wp2+4)*nc2
wfc=50
wfc2=10

### Graph

In [11]:
import tensorflow as tf
tf.reset_default_graph()
x = tf.placeholder('float', [None, rlat,rlon,1])
y = tf.placeholder('float',[None,1])
def convolutional_neural_network(x):
    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='VALID')
    def maxpool2d(x,W):
        return tf.nn.max_pool(x, [1,W,W,1], strides=[1,1,1,1], padding='VALID')
    weights = {'W_conv1':tf.Variable(tf.random_normal([wc1,wc1,1,nc1],stddev=0.0001,name="W_conv1")),
               'W_conv2':tf.Variable(tf.random_normal([wc2,wc2,nc1,nc2],stddev=0.0001)),
               'W_fc':tf.Variable(tf.random_normal([fl,wfc],stddev=0.0001)),
               'W_fc2':tf.Variable(tf.random_normal([wfc,wfc2],stddev=0.0001)),
               'out':tf.Variable(tf.random_normal([wfc2, 1],stddev=0.0001))}

    biases = { 'b_conv1':tf.Variable(tf.random_normal([nc1],stddev=0.001)),
               'b_conv2':tf.Variable(tf.random_normal([nc2],stddev=0.001)),
               'b_fc':tf.Variable(tf.random_normal([wfc],stddev=0.001)),
               'b_fc2':tf.Variable(tf.random_normal([wfc2],stddev=0.001)),
               'out':tf.Variable(tf.random_normal([1],stddev=0.001))}
    
    
    conv1 = conv2d(x, weights['W_conv1']) + biases['b_conv1']
    pool1 = maxpool2d(conv1,wp1)

    conv2 = conv2d(pool1, weights['W_conv2']) + biases['b_conv2']
    pool2 = maxpool2d(conv2,wp1)

    fc = tf.reshape(pool2,[-1, fl])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc2 = tf.nn.relu(tf.matmul(fc, weights['W_fc2'])+biases['b_fc2'])
    output = tf.matmul(fc2, weights['out'])+biases['out'] 
    return output

In [12]:
prediction = convolutional_neural_network(x)
cost = tf.reduce_mean(tf.square(prediction-y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
all_saver = tf.train.Saver()

# Train

In [13]:
def train_neural_network(x):
    batch=120
    epoches=300
    gtestcost=np.inf
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(epoches):
            for b in range(ptraining/batch):
                epoch_x=x_data[b*batch:(b+1)*batch,:,:,:]
                epoch_y=y_data[b*batch:(b+1)*batch,:]
                sess.run(optimizer, feed_dict={x: epoch_x, y: epoch_y})
                trainingcost=sess.run(cost, feed_dict={x: epoch_x, y: epoch_y})
                ctraining.append(trainingcost)
            testcost=sess.run(cost, feed_dict={x: x_data[ptraining:,:,:,:], 
                                                   y: y_data[ptraining:,:]})
            if testcost < gtestcost:
                gtestcost=testcost
        sess.close()
    return gtestcost

In [14]:
tf.set_random_seed(23)
tempt=train_neural_network(x)

In [7]:
globalbest=np.inf
for i in range(1000):
    tf.set_random_seed(i)
    tempt=train_neural_network(x)
    if tempt<globalbest:
        globalbest=tempt
        print [i,globalbest]

[0, 0.54207468]
[1, 0.36173707]
[2, 0.20643513]
[23, 0.17334057]


KeyboardInterrupt: 